## Load Package

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from pandas.errors import ParserError
import re

## 20211122 데이터에서만 Paser Error가 발생 -> 선 전처리
- 깃허브에 업로드한 데이터를 사용하면 실행x
- 추후, 데이콘에서 공공데이터셋으로 재구현할 때를 위해 구축해둠

In [327]:
# ParserError 데이터셋 처리
# 20111122 데이터만 오류가 있어, 이를 먼저 전처리
imp = pd.read_csv('CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_202111.csv', sep='\t', encoding='CP949')
imp

,"사용일자,""노선명"",""역명"",""승차총승객수"",""하차총승객수"",""등록일자"""
0,"20211101,""1호선"",""동대문"",""10855"",""10327"",""20211104"""
1,"20211101,""1호선"",""신설동"",""13933"",""13614"",""20211104"""
2,"20211101,""1호선"",""제기동"",""17584"",""17948"",""20211104"""
3,"20211101,""1호선"",""청량리(서울시립대입구)"",""19604"",""19787"",..."
4,"20211101,""2호선"",""시청"",""23006"",""22704"",""20211104"""
...,...
18077,"20211130,""중앙선"",""오빈"",""249"",""236"",""20211203"""
18078,"20211130,""중앙선"",""양평"",""2258"",""2229"",""20211203"""
18079,"20211130,""중앙선"",""원덕"",""260"",""247"",""20211203"""
18080,"20211130,""중앙선"",""용문"",""1610"",""1558"",""20211203"""


In [328]:
error_idx = []
right_txt = []

for idx, text in enumerate(imp['사용일자,"노선명","역명","승차총승객수","하차총승객수","등록일자"']):
    text = text.split(',')
    if len(text)!=6:
        error_idx.append(idx)
        text.pop(2)
        text = ', '.join(text)
        imp['사용일자,"노선명","역명","승차총승객수","하차총승객수","등록일자"'][idx] = text

In [343]:
imp.to_csv('CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_202111.csv', index=False)

In [2]:
imp = pd.read_csv('CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_202111.csv')

In [478]:
imp = imp['사용일자,"노선명","역명","승차총승객수","하차총승객수","등록일자"'].str.split(',', expand=True)

In [376]:
imp.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자']

In [499]:
# for col in imp.columns:
#     imp[col] = imp[col].astype(str)

In [500]:
imp['사용일자'] = imp['사용일자'].str.strip()
imp['노선명'] = imp['노선명'].str.strip()
imp['역명'] = imp['역명'].str.strip()
imp['승차총승객수'] = imp['승차총승객수'].str.strip()
imp['하차총승객수'] = imp['하차총승객수'].str.strip()
imp['등록일자'] = imp['등록일자'].str.strip()

In [501]:
def dequote(s):
    if (s[0] == s[-1]) and s.startswith(("'", '"')):
        return s[1:-1]
    return s

In [502]:
for col in imp.columns[1:]:
    imp[col] = imp[col].apply(lambda x: dequote(x))

In [503]:
imp['사용일자'] = imp['사용일자'].astype(int)
imp['승차총승객수'] = imp['승차총승객수'].astype(int)
imp['하차총승객수'] = imp['승차총승객수'].astype(int)
imp['등록일자'] = imp['등록일자'].astype(int)

In [519]:
imp

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20211101,1호선,동대문,10855,10855,20211104
1,20211101,1호선,신설동,13933,13933,20211104
2,20211101,1호선,제기동,17584,17584,20211104
3,20211101,1호선,청량리(서울시립대입구),19604,19604,20211104
4,20211101,2호선,시청,23006,23006,20211104
...,...,...,...,...,...,...
18077,20211130,중앙선,오빈,249,249,20211203
18078,20211130,중앙선,양평,2258,2258,20211203
18079,20211130,중앙선,원덕,260,260,20211203
18080,20211130,중앙선,용문,1610,1610,20211203


In [520]:
imp.to_csv('CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_202111.csv', index=False)

## Load Data: 지하철역별 일별 승하차 인원 정보

In [2]:
# 빈리스트
f = pd.DataFrame(columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'])

In [3]:
year = ['2018', '2019', '2020', '2021']
month =['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [4]:
for y in tqdm(year):
    for m in month:
        
        if  (y=='2020') & (m=='09'):
            df = pd.read_csv(f'CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_{y}{m}.csv', encoding='CP949').reset_index()
            df.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자', 'NA']
            df.drop('NA', axis=1, inplace = True)
            f = pd.concat([f, df], axis=0)
            continue
        
        elif (y=='2021') & (m=='11'):
            df = pd.read_csv(f'CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_{y}{m}.csv')
            f = pd.concat([f, df], axis=0)
            continue
        
        try:
            df = pd.read_csv(f'CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_{y}{m}.csv', encoding='CP949')
            f = pd.concat([f, df], axis=0)
            
        except UnicodeDecodeError:
            df = pd.read_csv(f'CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_{y}{m}.csv').reset_index()
            df.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자', 'NA']
            df.drop('NA', axis=1, inplace = True)
            f = pd.concat([f, df], axis=0)
            
        
        except ParserError or NameError:
            df = pd.read_csv(f'CARD_SUBWAY_MONTH/CARD_SUBWAY_MONTH_{y}{m}.csv', encoding='CP949', engine='c', error_bad_lines=False, warn_bad_lines=False).reset_index()
            f = pd.concat([f, df], axis=0)

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
f.reset_index(drop=True, inplace=True)

In [6]:
f.isna().sum()

사용일자      0
노선명       0
역명        0
승차총승객수    0
하차총승객수    0
등록일자      0
dtype: int64

In [7]:
f.drop(['등록일자'], axis=1, inplace=True)

In [8]:
f['사용일자'].nunique()

1461

## Load Data: 자치구별 지하철역 정보

In [9]:
gu_info = pd.read_csv('CARD_SUBWAY_MONTH/서울교통공사_자치구별지하철역정보_20220629.csv', encoding='CP949')
need_gu_list = pd.read_csv('sample_submission.csv').columns[1:]
need_gu_list

Index(['광진구', '동대문구', '성동구', '중랑구'], dtype='object')

In [10]:
need_gu_info = gu_info[gu_info['자치구'].isin(need_gu_list)].reset_index(drop=True)
need_gu_info.drop(['연번'], axis=1, inplace=True)
need_gu_info

,자치구,해당역(호선),역개수
0,광진구,"건대입구(2), 구의(2), 강변(2), 군자(5), 아차산(5), 광나루(5), ...",11
1,동대문구,"신설동(1), 제기동(1), 청량리(1), 용두(2), 신설동(2), 장한평(5)",6
2,성동구,"상왕십리(2), 왕십리(2), 한양대(2), 뚝섬(2), 성수(2), 용답(2), ...",14
3,중랑구,"봉화산(6), 신내(6), 먹골(7), 중화(7), 상봉(7), 면목(7), 사가정...",8


In [11]:
for idx, stations in enumerate(need_gu_info['해당역(호선)']):
    clean = []
    stations = stations.split(', ')
    
    for station in stations:
        clean.append(station[:-3])
    
    clean = list(set(clean))
    need_gu_info.at[idx, '해당역(호선)'] = clean
    need_gu_info.at[idx, '역개수'] = len(clean)

In [12]:
need_gu_info

,자치구,해당역(호선),역개수
0,광진구,"[군자, 뚝섬유원지, 강변, 건대입구, 광나루, 중곡, 구의, 아차산, 어린이대공원]",9
1,동대문구,"[장한평, 용두, 신설동, 청량리, 제기동]",5
2,성동구,"[왕십리, 뚝섬, 옥수, 용답, 신답, 신금호, 상왕십리, 마장, 금호, 한양대, ...",13
3,중랑구,"[용마산, 봉화산, 사가정, 상봉, 먹골, 중화, 면목, 신내]",8


## 필요한 지하철역 승하차인원 데이터만 가져오기

In [13]:
gwangjin_str = ''
for station in need_gu_info['해당역(호선)'][0]:
     gwangjin_str += station + '|'
gwangjin_str = gwangjin_str[:-1]
print(gwangjin_str)

군자|뚝섬유원지|강변|건대입구|광나루|중곡|구의|아차산|어린이대공원


In [14]:
dongdaemoon_str = ''
for station in need_gu_info['해당역(호선)'][1]:
     dongdaemoon_str += station + '|'
dongdaemoon_str = dongdaemoon_str[:-1]
print(dongdaemoon_str)

장한평|용두|신설동|청량리|제기동


In [15]:
seongdong_str = ''
for station in need_gu_info['해당역(호선)'][2]:
     seongdong_str += station + '|'
seongdong_str = seongdong_str[:-1]
print(seongdong_str)

왕십리|뚝섬|옥수|용답|신답|신금호|상왕십리|마장|금호|한양대|성수|답십리|행당


In [16]:
jungnang_str = ''
for station in need_gu_info['해당역(호선)'][3]:
     jungnang_str += station + '|'
jungnang_str = jungnang_str[:-1]
print(jungnang_str)

용마산|봉화산|사가정|상봉|먹골|중화|면목|신내


In [18]:
idx_list = []

for idx, station in tqdm(enumerate(f['역명'])):
    
    catch_g = re.findall(gwangjin_str, station)
    if len(catch_g)!=0:
        idx_list.append(idx)
        f.loc[idx, '자치구'] = '광진구'
        continue
    
    else:
        catch_d = re.findall(dongdaemoon_str, station)
        if len(catch_d)!=0:
            idx_list.append(idx)
            f.loc[idx, '자치구'] = '동대문구'
            continue
        
        else:
            catch_s = re.findall(seongdong_str, station)
            if len(catch_s)!=0:
                idx_list.append(idx)
                f.loc[idx, '자치구'] = '성동구'
                continue
            
            else:
                catch_j = re.findall(jungnang_str, station)
                if len(catch_j)!=0:
                    idx_list.append(idx)
                    f.loc[idx, '자치구'] = '중랑구'
                    continue

0it [00:00, ?it/s]

In [19]:
f

,사용일자,노선명,역명,승차총승객수,하차총승객수,자치구
0,20180101,1호선,서울역,40213.0,34505.0,NaN
1,20180101,우이신설선,신설동,699.0,809.0,동대문구
2,20180101,우이신설선,보문,754.0,803.0,NaN
3,20180101,우이신설선,성신여대입구(돈암),2228.0,2276.0,NaN
4,20180101,우이신설선,정릉,1844.0,1685.0,NaN
...,...,...,...,...,...,...
864847,20211231,6호선,월곡(동덕여대),10214.0,9924.0,NaN
864848,20211231,6호선,상월곡(한국과학기술연구원),5619.0,4842.0,NaN
864849,20211231,6호선,돌곶이,8265.0,7811.0,NaN
864850,20211231,6호선,석계,11189.0,11335.0,NaN


In [20]:
df = f[~f['자치구'].isna()].reset_index(drop=True)
df

,사용일자,노선명,역명,승차총승객수,하차총승객수,자치구
0,20180101,우이신설선,신설동,699.0,809.0,동대문구
1,20180101,7호선,뚝섬유원지,4558.0,4320.0,광진구
2,20180101,7호선,건대입구,9395.0,9772.0,광진구
3,20180101,7호선,어린이대공원(세종대),6171.0,6279.0,광진구
4,20180101,7호선,군자(능동),7034.0,4447.0,광진구
...,...,...,...,...,...,...
66531,20211231,우이신설선,신설동,1853.0,1786.0,동대문구
66532,20211231,경원선,옥수,39.0,30.0,성동구
66533,20211231,경원선,왕십리(성동구청),13743.0,15185.0,성동구
66534,20211231,경원선,청량리(서울시립대입구),15331.0,17470.0,동대문구


In [21]:
df.to_csv('pre_stepdata/subway_human.csv', index=False)

## 구별 일별 승차, 하차 총승객수

In [18]:
df = pd.read_csv('pre_stepdata/subway_human.csv')
df

,사용일자,노선명,역명,승차총승객수,하차총승객수,자치구
0,20180101,우이신설선,신설동,699.0,809.0,동대문구
1,20180101,7호선,뚝섬유원지,4558.0,4320.0,광진구
2,20180101,7호선,건대입구,9395.0,9772.0,광진구
3,20180101,7호선,어린이대공원(세종대),6171.0,6279.0,광진구
4,20180101,7호선,군자(능동),7034.0,4447.0,광진구
...,...,...,...,...,...,...
66531,20211231,우이신설선,신설동,1853.0,1786.0,동대문구
66532,20211231,경원선,옥수,39.0,30.0,성동구
66533,20211231,경원선,왕십리(성동구청),13743.0,15185.0,성동구
66534,20211231,경원선,청량리(서울시립대입구),15331.0,17470.0,동대문구


In [19]:
df_st = df.groupby(['자치구', '사용일자'])[['승차총승객수', '하차총승객수']].sum().reset_index()
df_st

,자치구,사용일자,승차총승객수,하차총승객수
0,광진구,20180101,132814.0,127828.0
1,광진구,20180102,220378.0,218082.0
2,광진구,20180103,223801.0,220743.0
3,광진구,20180104,224870.0,221870.0
4,광진구,20180105,240028.0,238971.0
...,...,...,...,...
5839,중랑구,20211227,118018.0,114456.0
5840,중랑구,20211228,123927.0,120249.0
5841,중랑구,20211229,127082.0,122922.0
5842,중랑구,20211230,124633.0,120390.0


In [20]:
df_st.to_csv('pre_stepdata/subway_human_gu_date.csv', index=False)

## 데이터셋 결합하기

In [54]:
train_df = pd.read_csv('data/train.csv')
train_df

,일시,광진구,동대문구,성동구,중랑구
0,20180101,0.592,0.368,0.580,0.162
1,20180102,0.840,0.614,1.034,0.260
2,20180103,0.828,0.576,0.952,0.288
3,20180104,0.792,0.542,0.914,0.292
4,20180105,0.818,0.602,0.994,0.308
...,...,...,...,...,...
1456,20211227,3.830,3.416,2.908,2.350
1457,20211228,4.510,3.890,3.714,2.700
1458,20211229,4.490,3.524,3.660,2.524
1459,20211230,4.444,3.574,3.530,2.506


In [55]:
# 학습데이터 구별로 나누기
df_g = train_df[['일시', '광진구']]
df_d = train_df[['일시', '동대문구']]
df_s = train_df[['일시', '성동구']]
df_j = train_df[['일시', '중랑구']]

df_g.shape, df_d.shape, df_s.shape, df_j.shape

((1461, 2), (1461, 2), (1461, 2), (1461, 2))

In [56]:
# 구별로 나눈 학습데이터 예시
df_g

,일시,광진구
0,20180101,0.592
1,20180102,0.840
2,20180103,0.828
3,20180104,0.792
4,20180105,0.818
...,...,...
1456,20211227,3.830
1457,20211228,4.510
1458,20211229,4.490
1459,20211230,4.444


In [57]:
# 승차, 하차 총승객수 구별로 나누기
g = df_st[df_st['자치구']=='광진구'].reset_index(drop=True)
d = df_st[df_st['자치구']=='동대문구'].reset_index(drop=True)
s = df_st[df_st['자치구']=='성동구'].reset_index(drop=True)
j = df_st[df_st['자치구']=='중랑구'].reset_index(drop=True)

g.shape, d.shape, s.shape, j.shape

((1461, 4), (1461, 4), (1461, 4), (1461, 4))

In [58]:
# 구별로 나눈 승차, 하차 총승객수
g

,자치구,사용일자,승차총승객수,하차총승객수
0,광진구,20180101,132814.0,127828.0
1,광진구,20180102,220378.0,218082.0
2,광진구,20180103,223801.0,220743.0
3,광진구,20180104,224870.0,221870.0
4,광진구,20180105,240028.0,238971.0
...,...,...,...,...
1456,광진구,20211227,167056.0,165106.0
1457,광진구,20211228,175166.0,172276.0
1458,광진구,20211229,180834.0,178342.0
1459,광진구,20211230,190152.0,186945.0


In [59]:
# 오류방지를 위한 데이터프레임 덮어쓰기
df_g = df_g.copy()
df_d = df_d.copy() 
df_s = df_s.copy() 
df_j = df_j.copy() 

In [60]:
# 피처 결합
df_g['승차총승객수'] = g['승차총승객수']
df_g['하차총승객수'] = g['하차총승객수']

df_d['승차총승객수'] = d['승차총승객수']
df_d['하차총승객수'] = d['하차총승객수']

df_s['승차총승객수'] = s['승차총승객수']
df_s['하차총승객수'] = s['하차총승객수']

df_j['승차총승객수'] = j['승차총승객수']
df_j['하차총승객수'] = j['하차총승객수']

In [63]:
df_g

,일시,광진구,승차총승객수,하차총승객수
0,20180101,0.592,132814.0,127828.0
1,20180102,0.840,220378.0,218082.0
2,20180103,0.828,223801.0,220743.0
3,20180104,0.792,224870.0,221870.0
4,20180105,0.818,240028.0,238971.0
...,...,...,...,...
1456,20211227,3.830,167056.0,165106.0
1457,20211228,4.510,175166.0,172276.0
1458,20211229,4.490,180834.0,178342.0
1459,20211230,4.444,190152.0,186945.0


In [67]:
df_g.to_csv('data/train_gwangjin.csv', index=False)
df_d.to_csv('data/train_dongdaemoon.csv', index=False)
df_s.to_csv('data/train_seongdong.csv', index=False)
df_j.to_csv('data/train_jungnang.csv', index=False)